In [30]:
# from master_lightsaber import lightsaber

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# from utils.uilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Fetch data

In [6]:
from ruamel.yaml import YAML
from Rohan import Cavalry

ImportError: No module named 'simplejson'

In [7]:
yaml = YAML()
yaml_file = './Rohan/cavalry.yml'
cav_info = open(yaml_file).read()
cav = yaml.load(cav_info)
cav

CommentedMap([('year', 2018),
              ('start_month', 2),
              ('end_month', 3),
              ('start_day', 20),
              ('end_day', 30),
              ('PC_data_dir',
               CommentedMap([('PC_first_dir', './Data/H/First/'),
                             ('PC_second_dir', './Data/H/Second/')])),
              ('Andriod_second_dir', './Data/A/Second/'),
              ('Terminal', 0),
              ('Scenario', [0, 1])])

In [10]:
cav['start_month'] = 6
cav['end_month'] = 6
cav['start_day'] = 1
cav['end_day'] = 30
cav['custid'] = 'lavada_positive'
# cav['custid'] = 'lavada_negative'
cav['Andriod_second_dir'] = './Data/A/'
cav['Terminal'] = 1
cav['appName'] = 'lavada_ta'
cav['test'] = 1

In [31]:
with open(yaml_file, "w") as f:
    yaml.dump(cav,f)

In [ ]:
# yaml = YAML()
# yaml_file = './Rohan/cavalry.yml'
# cav_info = open(yaml_file).read()
# cav = yaml.load(cav_info)
# cav

In [11]:
fetch_data_info = [cav['year'], cav['start_month'], cav['end_month'], cav['start_day'], cav['end_day'], cav['PC_data_dir'],
                  cav['Andriod_second_dir'], cav['Terminal'], cav['custid'], cav['appName'], cav['test']]

In [ ]:
test = Cavalry.RohanCavlry(fetch_data_info, 'table_of_fetch', 'verification', 'captcha')

In [12]:
b1_b, b2_b, local1, local2 = test.form()
# b1_h, b2_h, local1, local2 = test.form()

NameError: name 'test' is not defined

In [ ]:
print(len(b1_h), len(b2_h), len(local1), len(local2))

In [ ]:
mobile_hu = []
for ind in range(0, len(b2_h)):
    if b2_h[ind].loc[0, 'terminal'] == 1:
        mobile_hu.append(b2_h[ind])

## agument data

In [ ]:
b2_b = b2_b * 45

In [ ]:
for i in b2_h:
    i.loc[0, 'Bot'] = 0
for i in b2_b:
    i.loc[0, 'Bot'] = 1

## Preprocessing

In [13]:
def feature_normalize(dataset):
    mu = np.mean(dataset, axis = 0)
    sigma = np.std(dataset, axis = 0)
    return (dataset - mu) / sigma

In [14]:
def segment_signal(data, window_size = 400):
    segments = np.empty((0, window_size, 2))
    
    labels = np.empty((0))
    
    for ind in range(0, len(data)):
        print("ind", ind)
        diff_size = window_size - len(data[ind].op_x)
        if diff_size > 0:
            sample = np.zeros((diff_size,), dtype = data[ind].op_x.dtypes) + 0.001
            
            seg_x = np.concatenate((np.array[data[ind].op_x], sample), axis = 0)
            seg_y = np.concatenate((np.array[data[ind].op_y], sample), axis = 0)
        else:
            seg_x = data[ind].loc[0: window_size - 1, 'op_x']
            seg_y = data[ind].loc[0: window_size - 1, 'op_y']
        seg_x = feature_normalize(seg_x)
        seg_y = feature_normalize(seg_y)
        
        segments = np.vstack([segments, np.dstack([seg_x, seg_y])])
        
        data[ind]['Bot'] = data[ind]['Bot'].astype(float)
        Bot_prob = data[ind].loc[0, 'Bot']
        
        if data[ind].loc[0, 'scenario'] == 0:
            if Bot_prob >= 0.69:
                Bclass = 1
            else:
                Bclass = 0
        elif data[ind].loc[0, 'scenario'] == 1:
            if Bot_prob >= 0.58:
                Bclass = 1
            else:
                Bclass = 0
        else:
            print("error happen here, check scenario value")
        labels = np.append(labels, Bclass)
        
    return segments, labels
            

In [16]:
window_size = 400
channel = 2

def transformer(data, window_size, channel):
    segments, labels = segments_signal(data, window_size)
    
    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    reshape_segments = segments.reshape(len(segments), window_size, channel)
    
    return reshape_segments, labels

In [ ]:
training_set = []
training_set[0:0] = b2_h
training_set[0:0] = b2_b

In [ ]:
training_set_segments,training_set_labels = transformer(training_set, window_size, channel)
print("transform testing set")
# training_set_segments,training_set_labels = transformer(training_set, window_size, channel)

In [ ]:
type(training_set_segments)

In [ ]:
training_set_segments.shape

In [ ]:
train_set_segments[0:2, 0:1, :]

In [ ]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(training_set_segments, training_set_labels, 
                                               stratify = training_set_labels,
                                               random_state = 123)

In [ ]:
lab_tr

In [ ]:
type(X_tr[0], len(lab_tr))

In [ ]:
lab_tr

## Define & train network

In [24]:
tf.reset_default_graph()

In [25]:
lstm_size = 6
lstm_layers = 2
batch_size = 200
seq_len = 400
learning_rate = 0.0001
epochs = 1000

n_classes = 2
n_channels = 2

In [26]:
graph = tf.Graph()

with graph.as_default():
    with tf.device("/gpu:2"):
        inputs = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
        labels = tf.placeholder(tf.float32, [None, n_classes], name='labels')
        keep_prob = tf.placeholder(tf.float32, name='keep')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

In [27]:
with graph.as_default():
    with tf.device("/gpu:2"):
        lstm_data_input = tf.transpose(inputs, [1, 0, 2])
        lstm_in = tf.reshape(lstm_data_input, [-1, n_channels])
        
        lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None)
        lstm_in = tf.split(lstm_in, seq_len, 0)
        
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop]* lstm_layers)
        
        initial_state = cell.zero_state(batch_size, tf.float32)

In [28]:
with graph.as_default():
    with tf.device("/gpu:2"):
        outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32, initial_state=initial_state)
        
        logits = tf.layers.dense(outputs[-1], n_classes, name='logits')
        
        y_ = tf.nn.softmax(logits, name='y_')
        
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
        
        train_op = tf.train.AdamOptimizer(learning_rate)
        
        gradients = train_op.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
        optimizer = train_op.apply_gradients(capped_gradients)
        
        cor_pre = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(tf.cast(cor_pre, tf.float32), name='accuracy')

In [ ]:
if (os.path.exists('checkpoints') == False):
    !mkdir checkpoints

In [ ]:
!export CUDA_VISIBLE_DEVICE="0,1,2,3"

In [ ]:
os.environ["CUDA_VISIBLE_DEVICE"] = "0, 1, 2, 3"

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth = True

config.log_device_placement = True
config.allow_soft_placement = True

config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [ ]:
train_acc = []
train_loss = []

val_acc = []
val_loss = []

with graph.as_default():
    saver = tf.train.Saver()
    
with tf.Session(graph=graph, config=config) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    
    for ec in range(epochs):
        state = sess.run(initial_state)
        
        for x,y in get_batches(X_tr, lab_tr, batch_size):
            feed = {inputs: x, labels: y, keep_prob: 0.5, initial_state: state, learning_rate: learning_rate}
            
            loss ,_ ,state, acc = sess.run([cost, optimizer, final_state, accuracy], feed_dict=feed)
            
            train_acc.append(acc)
            train_loss.append(loss)
            
            if(iteration %5 == 0):
                print("Epoch:{}/{}".format(ec, epochs), "Iteration:{:d}".format(iteration), "Train loss:{:.6f}".format(loss, acc))
                
            if(iteration %25 == 0):
                
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                val_acc = []
                val_loss = []
                
                for x_v, y_v in get_batches(X_vld, lab_vld, batch_size):
                    feed = {inputs: x_v, labels: y_v, keep_prob: 1.0, initial_state: val_state}
                    
                    loss_v ,state_v, acc_v = sess.run([cost, final_state, accuracy], feed_dict=feed)
                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                print("Epoch:{}/{}".format(ec, epochs), 
                      "Iteration:{:d}".format(iteration),
                      "Validation loss:{:.6f}".format(np.mean(val_loss_)),
                      "Validation acc:{:.6f}".format(np.mean(val_acc_)))
                
                val_acc.append(np.mean(val_acc_))
                val_loss.append(np.mean(val_loss_))
                
            iteration += 1
        saver.save(sess, "checkpoints/gandalf_lstm.ckpt")
    saver.save(sess, "checkpoints/gandalf_lstm.ckpt")

In [ ]:
t = np.arange(iteration - 1)

plt.figure(figsize=(14, 8))

plt.plot(t, np.array(train_loss), 'r-', t[t % 25 == 0], np.array(val_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(t, np.array(train_acc), 'r-', t[t % 25 == 0], np.array(val_acc), 'b*')
plt.xlabel("Iteration")
plt.ylabel("Acc")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
!nvidia-smi